In [1]:
import sys
sys.path.append("../../smsl")

In [2]:
from smsl.graphBigtraj import BigTrajGraphAgent
from smsl.config import ConfAgent
from smsl.graph import BM

In [3]:
import os
import pandas as pd
import numpy as np
from IPython.display import JSON

In [4]:
class HotSpotAgent(dict, ConfAgent):
    def __init__(self, MAgent):
        ConfAgent.__init__(self)
        self.end_ns = self.time_num
        self.framesperns = self.frame_num // self.time_num
        self.MAgent = MAgent
    def load_systems(self, system2offical_name):
        ori_path = os.getcwd()
        for system, system_offical_name in system2offical_name.items():
            os.chdir(f'../../{system}/d.fluctmatch')
            b_agent = BigTrajGraphAgent(self.MAgent)
            b_agent.system_offical_name = system_offical_name
            self[system] = b_agent
        os.chdir(ori_path)
    def get_hotspot(self, verbose=0, is_return=False):
        self.m_abbr = self.get_m_abbr() 
        dict_hotspot = {self.m_abbr:{}}
        for system, b_agent in self.items():
            dict_hotspot[b_agent.m_abbr][system] = {}
            dict_hotspot[b_agent.m_abbr][system] = get_hotspot_by_graph_agent(b_agent.t_agent, verbose=verbose)
        if is_return:
            return dict_hotspot
        else:
            self.dict_hotspot = dict_hotspot
    def get_m_abbr(self):
        return list(self.values())[0].m_abbr ## first system of sub group name
    def get_df_k_with_hotspots(self, k_value='k_avg_std', is_return=False): ## for s = bm
        list_k_with_hotspots = []
        for s, dict_subgroup in self.dict_hotspot.items(): 
            for system, dict_system in dict_subgroup.items():
                for position, dict_position in dict_system.items():
                    pos_col, pos_row_2, pos_row_1  = position[0], position[1], position[2]
                    k_avg = dict_position['hotspot']['avg']['avg']
                    k_std = dict_position['hotspot']['avg']['std']
                    list_k_with_hotspots.append({
                        'system'   : self[system].system_offical_name,
                        'metal'    : pos_row_1,
                        'layer'    : pos_row_2,
                        'gstrand'  : pos_col,
                        'k_avg'    : k_avg,
                        'k_std'    : k_std,
                        'k_avg_std': f'{k_avg:.2f} ± {k_std:.2f}',
                    })
        df_k_with_hotspots_1 = pd.DataFrame(list_k_with_hotspots)
        df_k_with_hotspots = df_k_with_hotspots_1.set_index(['system', 'metal', 'layer'])
        df_k_with_hotspots = df_k_with_hotspots.pivot(columns='gstrand', values=k_value)

        ## sort the row and column index
        sort_system  = list(dict.fromkeys(df_k_with_hotspots_1['system']))
        sort_ml      = list(dict.fromkeys(map(tuple, df_k_with_hotspots_1[['metal', 'layer']].values)))
        sort_gstrand = list(dict.fromkeys(df_k_with_hotspots_1['gstrand']))
        sort_row = [[system, metal, layer] for system in sort_system for metal, layer in sort_ml]
        df_k_with_hotspots = df_k_with_hotspots.loc[sort_row, sort_gstrand]
        if is_return:
            return df_k_with_hotspots
        else:
            self.df_k_with_hotspots = df_k_with_hotspots 

In [5]:
def generate_cluster_hotspots(self): ## for s = bm
    metal_name = self.mean.df_m['Atomname_j'].values[0]
    cluster2hotspots = {
        'all': {1: ['O6', metal_name]},
    }
    return cluster2hotspots

## SmallTimeAgent level
def get_cluster(self, resid_i, resid_j): ## for s = bm
    return 'all'

## SmallTimeAgent level, for s = bm
def generate_df_position2hotspots(self, resid_i, resid_j, hotspots): ## for s = bm
    dict_ks = {}
    df_m = self.mean.df_m
    mask1 = (df_m['Resid_i']==resid_i) & (df_m['Resid_j']==resid_j)
    df_resid_ij = df_m[mask1]
    for i, hotspot in hotspots.items():
        if isinstance(hotspot[1], int): ## rank n: ['rank', 1]
            df_sele = df_resid_ij.sort_values(by='k', ascending=False).iloc[[hotspot[1]-1]]
        elif isinstance(hotspot[1], str): ## hotspot: ['N1', 'O6']
            mask2 = (df_resid_ij['Atomname_i']==hotspot[0]) & (df_resid_ij['Atomname_j']==hotspot[1])
            df_sele = df_resid_ij[mask2]
        else:
            raise RuntimeError('Could not recognize cluster2hotspots:', cluster2hotspots)
        ## generate dict_ks
        pair_id, ser_sele = df_sele.index, df_sele.iloc[0]
        dict_ks[i] = {
            'query'     : ''.join(map(str, hotspot)),
            'atomname_i': ser_sele.Atomname_i,
            'atomname_j': ser_sele.Atomname_j,
            'k'         : {},
        }
        for time_label, st_agent in self.items():
            dict_ks[i]['k'][time_label] = st_agent.df_m.loc[pair_id].squeeze().k
        ks = pd.Series(dict_ks[i]['k'])
        dict_ks[i]['k']['avg'] = ks.mean()
        dict_ks[i]['k']['std'] = ks.std() ## Note: numpy.std(ddof=0), but pandas.std(ddof=1)
    ## k_avg in {aiaj}^s
    dict_ks['avg']  = {}
    for time_label, _ in self.items():
        ks = []
        for i, hotspot in hotspots.items():
            ks.append(dict_ks[i]['k'][time_label])
        dict_ks['avg'][time_label] = np.mean(ks)
    ks = pd.Series(dict_ks['avg'])
    dict_ks['avg']['avg'] = ks.mean()
    dict_ks['avg']['std'] = ks.std() ## Note: numpy.std(ddof=0), but pandas.std(ddof=1)
    return dict_ks

## GraphAgent level, for s = bm
def get_hotspot_by_graph_agent(self, verbose=0):  ## for s = hb
    gstrands_i = self.df_tetrad_geometry.columns.values
    layers_ij2metals   = {'top': ['K1'], 'mid':['K1','K2'], 'bot':['K2']}
    cluster2hotspots = generate_cluster_hotspots(self)
    dict_position = {}
    for layer, metals in layers_ij2metals.items():
        for metal in metals:
            for gstrand_i in gstrands_i:
                resid_i = self.df_tetrad_geometry.loc[layer,gstrand_i]
                resid_j = 3 - int(metal[1]) ## Note: K1(resid_j=2, on top), K2(resid_j=1, on bot)
                ## generate dict_hotspot
                cluster = get_cluster(self.mean, resid_i, resid_j)
                hotspots = cluster2hotspots[cluster] 
                if verbose:
                    print(self.system, metal, layer, gstrand_i, resid_i, resid_j, cluster, hotspots)
                dict_ks = generate_df_position2hotspots(self, resid_i, resid_j, hotspots)
                dict_position[(gstrand_i, layer, metal)] = {
                    'resid_i': int(resid_i), ## save json format from int istead of np.int64(cannot save) 
                    'resid_j': int(resid_j),
                    'cluster': cluster,
                    'hotspot': dict_ks
                }
    return dict_position

In [6]:
system2offical_name = {
'propeller': 'Propeller',
'basket'   : 'Basket',
'chair'    : 'Chair',
'hybrid-i' : 'Hybrid-I',
'hybrid-ii': 'Hybrid-II',
# 'dsdna'    : 'dsDNA',
}

In [7]:
hs_agent = HotSpotAgent(BM)
hs_agent.load_systems(system2offical_name)

Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/0000_0500/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/0250_0750/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/0500_1000/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/0750_1250/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/1000_1500/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/1250_1750/result/pairtypes_k_b0_cutoff_5.00.csv


/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to 

Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/1500_2000/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/1750_2250/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/2000_2500/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/2250_2750/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/2500_3000/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/2750_3250/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fl

/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to 

Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/3500_4000/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/3750_4250/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/4000_4500/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/4250_4750/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/4500_5000/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/0000_5000/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/basket/d.fluctmatch/bigtraj_fluct

...

/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to 

Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii/d.fluctmatch/bigtraj_fluctmatch/1250_1750/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii/d.fluctmatch/bigtraj_fluctmatch/1500_2000/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii/d.fluctmatch/bigtraj_fluctmatch/1750_2250/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii/d.fluctmatch/bigtraj_fluctmatch/2000_2500/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii/d.fluctmatch/bigtraj_fluctmatch/2250_2750/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii/d.fluctmatch/bigtraj_fluctmatch/2500_3000/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii/d.fluctmatch/bigtraj_fl

/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to 

Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii/d.fluctmatch/bigtraj_fluctmatch/3250_3750/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii/d.fluctmatch/bigtraj_fluctmatch/3500_4000/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii/d.fluctmatch/bigtraj_fluctmatch/3750_4250/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii/d.fluctmatch/bigtraj_fluctmatch/4000_4500/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii/d.fluctmatch/bigtraj_fluctmatch/4250_4750/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii/d.fluctmatch/bigtraj_fluctmatch/4500_5000/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii/d.fluctmatch/bigtraj_fl

/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))


In [8]:
hs_agent.get_hotspot()
hs_agent.get_df_k_with_hotspots()

In [9]:
JSON(hs_agent.dict_hotspot)

<IPython.core.display.JSON object>

/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/jupyter_client/session.py:719: UserWarning: Message serialization failed with:
keys must be str, int, float, bool or None, not tuple
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


In [10]:
## K1(top), K2(bot)
hs_agent.df_k_with_hotspots

gstrand                          Q3            Q2            Q1            Q4
system    metal layer                                                        
Propeller K1    top    16.30 ± 0.83  10.38 ± 2.64  13.13 ± 1.54  14.09 ± 1.81
                mid     5.69 ± 1.91   4.34 ± 1.83   7.15 ± 3.32   1.23 ± 1.42
          K2    mid     3.88 ± 1.89   3.37 ± 1.02   4.13 ± 1.54   1.40 ± 1.59
                bot     9.95 ± 0.65  11.81 ± 1.34  10.23 ± 2.20  13.52 ± 2.09
Basket    K1    top    18.19 ± 0.74   4.20 ± 0.57  10.04 ± 0.50  16.55 ± 0.76
                mid     4.91 ± 0.65  10.02 ± 0.61   6.36 ± 0.66   0.00 ± 0.00
          K2    mid     3.98 ± 0.31   4.92 ± 0.69   1.36 ± 0.40   4.66 ± 0.50
                bot     8.68 ± 1.00  12.67 ± 1.15  12.88 ± 2.26  16.18 ± 1.03
Chair     K1    top    13.43 ± 0.58  20.25 ± 0.63  11.32 ± 0.79  18.21 ± 0.52
                mid     5.84 ± 0.45   1.54 ± 0.37   9.42 ± 1.01   0.70 ± 0.36
          K2    mid     4.69 ± 1.49   8.92 ± 2.34  13.15 ± 4.67   9.94 ± 1.02
                bot     4.86 ± 4.26  27.53 ± 0.76   0.00 ± 0.00  24.40 ± 1.13
Hybrid-I  K1    top    10.64 ± 1.01  10.41 ± 0.34  18.41 ± 1.26  14.49 ± 1.25
                mid     8.99 ± 1.47   9.14 ± 1.45   0.49 ± 1.08   6.73 ± 1.07
          K2    mid     6.20 ± 1.09   8.50 ± 1.06   0.57 ± 0.33   0.05 ± 0.08
                bot     7.08 ± 0.52   8.03 ± 0.63  21.96 ± 0.87  12.34 ± 1.32
Hybrid-II K1    top    15.96 ± 2.29  18.67 ± 0.53  12.80 ± 0.89  14.06 ± 1.59
                mid     5.24 ± 0.41   0.03 ± 0.08  11.01 ± 1.63   0.47 ± 0.81
          K2    mid     5.35 ± 1.92   0.00 ± 0.00   1.59 ± 0.89   3.69 ± 0.70
                bot     8.09 ± 2.06  21.39 ± 0.33   4.45 ± 2.52  14.40 ± 1.82

In [11]:
system2offical_name = {
'propeller': 'Propeller',
'basket'   : 'Basket',
'chair'    : 'Chair',
'hybrid-i' : 'Hybrid-I',
'hybrid-ii': 'Hybrid-II',
# 'dsdna'    : 'dsDNA',
'basket_na': 'Basket-Na',
'hybrid-ii_wttel26'    : 'Hybrid-II-wtTel26',
'propeller_poh'        : 'Propeller-POH',
'propeller_ewv'        : 'Propeller-EWV', 
'hybrid-ii_wttel26_ewv': 'Hybrid-II-wtTel26-EWV', 
'hybrid-ii_wttel26_poh': 'Hybrid-II-wtTel26-POH', 
}

In [12]:
hs_agent = HotSpotAgent(BM)
hs_agent.load_systems(system2offical_name)
hs_agent.get_hotspot()
hs_agent.get_df_k_with_hotspots()

/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to 

Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/0000_0500/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/0250_0750/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/0500_1000/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/0750_1250/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/1000_1500/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/1250_1750/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fl

/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to 

Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/2500_3000/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/2750_3250/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/3000_3500/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/3250_3750/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/3500_4000/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fluctmatch/3750_4250/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/propeller/d.fluctmatch/bigtraj_fl

/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to 

...

Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii_wttel26_poh/d.fluctmatch/bigtraj_fluctmatch/0500_1000/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii_wttel26_poh/d.fluctmatch/bigtraj_fluctmatch/0750_1250/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii_wttel26_poh/d.fluctmatch/bigtraj_fluctmatch/1000_1500/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii_wttel26_poh/d.fluctmatch/bigtraj_fluctmatch/1250_1750/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii_wttel26_poh/d.fluctmatch/bigtraj_fluctmatch/1500_2000/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii_wttel26_poh/d.fluctmatch/bigtraj_fluctmatch/1750_2250/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /ho

/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to 

Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii_wttel26_poh/d.fluctmatch/bigtraj_fluctmatch/2500_3000/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii_wttel26_poh/d.fluctmatch/bigtraj_fluctmatch/2750_3250/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii_wttel26_poh/d.fluctmatch/bigtraj_fluctmatch/3000_3500/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii_wttel26_poh/d.fluctmatch/bigtraj_fluctmatch/3250_3750/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii_wttel26_poh/d.fluctmatch/bigtraj_fluctmatch/3500_4000/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii_wttel26_poh/d.fluctmatch/bigtraj_fluctmatch/3750_4250/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /ho

/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/user/anaconda3/envs/fluctmatch/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to 

Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii_wttel26_poh/d.fluctmatch/bigtraj_fluctmatch/4250_4750/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii_wttel26_poh/d.fluctmatch/bigtraj_fluctmatch/4500_5000/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii_wttel26_poh/d.fluctmatch/bigtraj_fluctmatch/0000_5000/result/pairtypes_k_b0_cutoff_5.00.csv


In [13]:
hs_agent.df_k_with_hotspots

gstrand                                      Q3            Q2            Q1  \
system                metal layer                                             
Propeller             K1    top    16.30 ± 0.83  10.38 ± 2.64  13.13 ± 1.54   
                            mid     5.69 ± 1.91   4.34 ± 1.83   7.15 ± 3.32   
                      K2    mid     3.88 ± 1.89   3.37 ± 1.02   4.13 ± 1.54   
                            bot     9.95 ± 0.65  11.81 ± 1.34  10.23 ± 2.20   
Basket                K1    top    18.19 ± 0.74   4.20 ± 0.57  10.04 ± 0.50   
                            mid     4.91 ± 0.65  10.02 ± 0.61   6.36 ± 0.66   
                      K2    mid     3.98 ± 0.31   4.92 ± 0.69   1.36 ± 0.40   
                            bot     8.68 ± 1.00  12.67 ± 1.15  12.88 ± 2.26   
Chair                 K1    top    13.43 ± 0.58  20.25 ± 0.63  11.32 ± 0.79   
                            mid     5.84 ± 0.45   1.54 ± 0.37   9.42 ± 1.01   
                      K2    mid     4.69 ± 1.49   8.92 ± 2.34  13.15 ± 4.67   
                            bot     4.86 ± 4.26  27.53 ± 0.76   0.00 ± 0.00   
Hybrid-I              K1    top    10.64 ± 1.01  10.41 ± 0.34  18.41 ± 1.26   
                            mid     8.99 ± 1.47   9.14 ± 1.45   0.49 ± 1.08   
                      K2    mid     6.20 ± 1.09   8.50 ± 1.06   0.57 ± 0.33   
                            bot     7.08 ± 0.52   8.03 ± 0.63  21.96 ± 0.87   
Hybrid-II             K1    top    15.96 ± 2.29  18.67 ± 0.53  12.80 ± 0.89   
                            mid     5.24 ± 0.41   0.03 ± 0.08  11.01 ± 1.63   
                      K2    mid     5.35 ± 1.92   0.00 ± 0.00   1.59 ± 0.89   
                            bot     8.09 ± 2.06  21.39 ± 0.33   4.45 ± 2.52   
Basket-Na             K1    top     3.14 ± 4.79   0.00 ± 0.00   0.00 ± 0.00   
                            mid     9.52 ± 8.15  19.77 ± 4.26  19.89 ± 9.59   
                      K2    mid     0.00 ± 0.00   0.30 ± 0.39   0.50 ± 0.51   
                            bot     6.32 ± 4.17  15.47 ± 5.33  11.92 ± 4.35   
Hybrid-II-wtTel26     K1    top    13.86 ± 0.36  20.18 ± 0.30  12.16 ± 0.23   
                            mid     7.32 ± 1.32   0.00 ± 0.00  15.64 ± 1.27   
                      K2    mid     5.60 ± 1.41   0.00 ± 0.00   8.97 ± 1.54   
                            bot     9.42 ± 0.56  21.44 ± 0.49   6.22 ± 0.38   
Propeller-POH         K1    top    14.99 ± 2.43  15.76 ± 1.05  10.72 ± 1.42   
                            mid     5.65 ± 1.55   6.68 ± 3.22   8.36 ± 2.12   
                      K2    mid     1.82 ± 2.03   2.63 ± 1.68   2.89 ± 2.32   
                            bot    11.45 ± 1.79  10.80 ± 2.21   6.75 ± 1.88   
Propeller-EWV         K1    top    14.35 ± 2.18  15.22 ± 1.45  14.89 ± 1.10   
                            mid    10.83 ± 1.89   5.07 ± 3.96   7.79 ± 2.55   
                      K2    mid     2.28 ± 1.43   1.01 ± 1.01   2.65 ± 1.30   
                            bot     9.58 ± 0.96  13.53 ± 3.37  11.47 ± 1.72   
Hybrid-II-wtTel26-EWV K1    top     7.51 ± 1.39   7.19 ± 1.65   9.20 ± 0.64   
                            mid     7.73 ± 4.20   3.75 ± 2.35  12.91 ± 4.06   
                      K2    mid     0.00 ± 0.00   0.00 ± 0.02   0.00 ± 0.00   
                            bot     6.90 ± 3.12  16.41 ± 7.29   6.07 ± 2.73   
Hybrid-II-wtTel26-POH K1    top    12.70 ± 2.28   3.68 ± 2.53   7.71 ± 1.60   
                            mid     7.61 ± 1.77   2.84 ± 2.29  13.86 ± 1.00   
                      K2    mid     0.01 ± 0.04   0.01 ± 0.03   0.00 ± 0.00   
                            bot     4.87 ± 4.08  12.63 ± 7.81   4.93 ± 3.50   

gstrand                                      Q4  
system                metal layer                
Propeller             K1    top    14.09 ± 1.81  
                            mid     1.23 ± 1.42  
                      K2    mid     1.40 ± 1.59  
                            bot    13.52 ± 2.09  
Basket                K1    top    16.55 ± 0.76  
               

### Save all k value data in json format

In [14]:
import json, copy
for subgroup, data in hs_agent.dict_hotspot.items():
    data_copy = copy.deepcopy(data)
    for system, d1 in data_copy.items():
        d1_keys = list(d1.keys())
        for position in d1_keys:
            if isinstance(position, tuple):
                link_position = ','.join(position)
                d1[link_position] = d1.pop(position)
    with open(f'data/{subgroup}.json', 'w') as f:
        json.dump(data_copy, f)

### Save k value with specific position in csv file

In [15]:
df_ks_position = hs_agent.get_df_k_with_hotspots(k_value='k_avg', is_return=True)
df_ks_position.to_csv(f'data/{hs_agent.m_abbr}_avg.csv')

In [16]:
df_ks_position = hs_agent.get_df_k_with_hotspots(k_value='k_std', is_return=True)
df_ks_position.to_csv(f'data/{hs_agent.m_abbr}_std.csv')